In [1]:
!pip install sentence_transformers
!pip install konlpy

In [2]:
import numpy as np
import itertools

from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from krwordrank.word import summarize_with_keywords

In [3]:
hannanum = Hannanum()

min_count = 2   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
beta = 0.90    # PageRank의 decaying factor beta
max_iter = 20
#한국어 불용어
stop_words="아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')

# SBERT를 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [4]:
doc_with_newlines = """
[IT동아 남시현 기자] 지난 3월 31일, 인텔이 아크(Arc) 그래픽 카드 라인업을 공개하며 그래픽 카드 사업 진출을 선언했다. 2018년 6월에 외장 그래픽 카드 개발 여부를 공개한 지 4년 만의 결과다. 인텔의 외장 그래픽 카드는 인텔 Xe 마이크로 아키텍처를 기반으로 하며, 노트북용 그래픽 카드를 시작으로 올해 여름까지 데스크톱 제품까지 차례로 공개한다. 이로써 지난 20년 간 엔비디아와 AMD가 나눠갖던 그래픽 카드 시장은 이제 세 개 기업의 경쟁 구도로 경쟁하게 된다.가장 처음 공개된 제품은 삼성전자 갤럭시 북2 프로에 탑재된 인텔 아크 A350M이다. A350M은 전체 노트북 그래픽 카드 라인업 중 가장 보급형인 제품이며, A350M을 시작으로 A370M, A550M, A730M, A770M의 순서로 성능이 향상된다. A 시리즈는 기존 GPU 벡터 장치 대비 16배가 향상된 인공지능 가속 기능 MXM AI 엔진과 최대 8K60p 12bit HDR 디코딩과 8K 10bit HDR 인코딩을 지원하는 Xe 미디어 엔진, 최대 두 대의 8K60p HDR 및 네 대의 4K 120p HDR 영상을 출력하는 Xe HPG 디스플레이 엔진을 탑재한다. 게이밍 역시 인공지능 기반의 해상도 향상 기능인 XeSS, 실시간 광선 추적 등이 적용된다. 갤럭시 북2 프로를 토대로 자세한 성능을 알아보자.인텔 아크 A350M은 인텔 아크 A 시리즈 그래픽 카드 중 가장 보급형 그래픽카드로, 6개의 Xe 코어와 6개의 레이 트레이싱(실시간 광선 추적) 유닛, 4GB의 GDDR6 메모리가 적용된다. 노트북용 그래픽 카드 중 가장 최상급인 A770M은 32개의 Xe 코어 및 레이 트레이싱 유닛, 16GB의 GDDR6 메모리를 갖추고 있다. 그래픽 카드의 전력은 25~35W 수준으로 게이밍 노트북보다는 고성능 경량 노트북에 적합하지만, H265와 AV1, VP9에 대한 하드웨어 인코딩 및 디코딩을 지원해 가벼운 게임 혹은 영상 편집 용도로 적합하다.컴퓨터의 프로세서 및 그래픽 칩셋 스펙을 확인하는 CPU-Z(좌측) 및 GPU-Z(우측)를 활용해 삼성전자 갤럭시 북2 프로의 스펙을 확인했다. 해당 노트북은 28W 프로세서인 12세대 인텔 코어 i5-1240P를 탑재하며, 그래픽 카드는 인텔 아크 A350M으로 확인된다. 그래픽 카드의 연산 성능을 짐작할 수 있는 픽셀 필레이트는 초당 70.4기가 픽셀, 3D화 하는 텍스처 필레이트는 초당 140.8기가 텍셀로 엔비디아 지포스 GTX 1660 Ti 모바일과 거의 비슷하다.그래픽 드라이버는 ‘인텔 아크 컨트롤’이라는 독자 소프트웨어로 구동된다. 그래픽 드라이버는 그래픽 카드의 기능 및 성능 관리, 게임 최적화 및 호환성 등을 위한 소프트웨어로, 작성일 기준 최신 버전은 30.0.101.1330 버전이다. 아크 컨트롤에서는 그래픽 드라이버의 최신 기능에 대한 소개와 지원 게임 연결, 그래픽 카드 온도 및 상황 등을 확인할 수 있는 퍼포먼스, 그리고 라이브 스트리밍 보조 기능인 스튜디오 기능이 제공된다. 자주 사용하는 기능은 위젯으로 정렬할 수 있고, 설정을 통해 서비스 설정 등을 바꿀 수 있다. 특히 엔비디아 지포스 익스피리언스와 달리 로그인하지 않고도 기능이 제공된다.생산성과 게이밍 성능은 어떤 수준? 인텔 아크 A350M의 성능을 확인해보고자 생산성 성능을 점수로 환산하는 프로그램, PC마크10 벤치마크를 활용해 성능을 측정해봤다. PC마크10은 앱 실행 속도나 화상 회의, 웹 브라우징, 액셀 및 워드 문서 편집, 사진 및 영상 편집, 3D 렌더링 등의 작업을 수행해 CPU 및 그래픽 카드의 작업 성능을 점수로 평가한다. PC마크 10을 활용해 확인한 인텔 i5-1240P 및 인텔 아크 A350M의 종합 점수는 5천355점, 그래픽 카드의 영향을 받는 디지털 콘텐츠 생산성 점수는 6천813점으로 확인된다. 엔비디아 지포스 MX450보다는 1천800점가량 높고, AMD 라데온 RX 5300M과 비슷한 점수다. 보급형 라인업인 만큼 RTX 3050보다는 낮지만, 초경량 노트북에 탑재된 그래픽 카드라는 점을 감안하면 좋은 점수다.인텔 아크 시리즈의 또 다른 특징은 영상 편집 성능이다. 인텔 아크의 Xe 미디어 엔진은 8K HDR 인코딩 및 디코딩을 포함해 H265, AV1, VP9 등 최신 코덱을 대거 지원한다. 또한 인텔 CPU의 내장 그래픽과 아크 그래픽 카드가 동시에 동작하는 하이퍼 인코딩 기능을 활용해 작업 효율을 끌어올리고 있다. 동영상 변환 프로그램인 ‘핸드브레이크’를 활용해 2분 13초 길이의 H.264 4K 30프레임, 비트레이트 60Mbps의 영상을 동일 크기의 H.265 영상으로 변환했다. 인텔 i5-1240P의 CPU만 활용한 편집에 걸린 시간은 10분 40초인데, 인텔 아이리스 Xe 내장 그래픽과 인텔 아크 A350M이 하이퍼 인코딩으로 동작했을 때 걸린 시간은 44초에 불과했다. 영상 편집 효율은 동급 구성의 타사 제품보다 좋을 것으로 판단된다.게이밍 성능을 확인하기에 앞서, 게이밍 성능을 점수로 환산하는 프로그램인 3D 마크의 타임스파이, 파이어 스트라이크를 각각 실행해 점수를 확인했다. 아이리스 Xe 내장 그래픽의 점수는 1천352점일 때, 인텔 아크 A350M의 점수는 2천976점으로 확인된다. 엔비디아 지포스 MX450의 타임스파이 점수가 1천900점 대, 지포스 GTX 1650 모바일의 타임스파이 점수가 3천500대이므로, 경량 노트북으로는 높은 편이다. 다이렉트X 11 기반 테스트인 파이어 스트라이크 점수는 7천287점으로, GTX1650 모바일의 8천 점과 비슷한 수준이다. 게이밍 성능은 라이즈 오브 더 툼레이더와 토탈 워 사가:트로이, 토탈워:워해머 II, 레드 데드 리뎀션 II, GTA V까지 다양하게 수행했다. 라이즈 오브 더 툼레이더는 2016년 출시된 게임으로, 권장 사양은 지포스 GTX970과 AMD 라데온 R9 290X다. 테스트는 FHD 해상도, 중간 옵션으로 진행했다. 이때 종합 점수는 41.04FPS로 약간은 끊어진다고 느낄 수준이다. 다행히 30프레임 이상은 확보하므로 게임 자체를 즐기는 데는 무리가 없으며, 사양을 낮추면 60프레임도 가능한 정도다. CPU 성능도 함께 중요한 토탈 워 사가:트로이와 토탈워:워해머 II를 각각 실행해봤다. 두 게임 모두 GTX 970 혹은 R9 290X 정도는 사용해야 한다. 먼저 토탈 워 사가: 트로이는 ‘높음’ 성능을 설정한 다음 배틀 벤치마크를 수행했다. 이때 인텔 아크 A350M의 프레임은 27.7fps로 무난한 수준이다. 반면 토탈워:워해머 II는 높음 설정에서 22.8fps로 다소 사양을 낮출 필요가 있다. 두 게임 모두 프로세서의 영향을 많이 받는 게임이지만, 옵션 타협으로 충분히 게임을 즐길 수준으로 볼 수 있다. 이보다 높은 사양의 CPU와 GPU를 조합하면 토탈워:삼국이나 토탈워:워해머 III는 무난하게 즐길 수 있을 것으로 보인다. 레드 데드 리뎀션 II는 720p 중간 옵션으로 설정해야 무난하게 플레이할 수 있다. 마지막으로 오픈월드 게임인 레드 데드 리뎀션II와 GTA V를 각각 실행했다. 레드 데드 리뎀션 II는 권장 사양이 GTX 1060 6GB, 라데온 RX 480 4G로 높은 게임이다. 그렇다 보니 FHD 해상도 보통 사양에서도 평균 22프레임으로 나타났다. 이 정도 사양의 게임을 플레이한다면 HD급 해상도로 조정해야 한다. 반면 권장 사양이 GTX 660 2GB 및 라데온 HD 7870으로 낮은 GTA V는 FHD 해상도 벤치마크에서 평균 77.1프레임, 최소 27.5프레임으로 안정적인 모습을 보여주었다. 배틀그라운드나 디아블로 2: 레저렉션 등의 게임도 중간 수준의 옵션이면 60프레임 내외로 즐길 수 있을 정도다. 인텔 아크 그래픽 카드는 인텔이 20여 년 만에 다시 공개한 그래픽 카드다. 완전히 새롭게 설계된 만큼 최신 코덱 지원과 세련된 공정으로 제작됐고, 인텔 아크 A350M으로만 확인한 정보만으로도 엔비디아는 어려워도 AMD는 긴장해야 할 수준이다. 다만, 그래픽 카드 생태계가 이제 막 시작한 터라 호환성이 부족한 점은 어쩔 수 없다. 테스트를 진행하는 과정에서도 3D 렌더링 벤치마크인 블랜더, V-레이가 그래픽 카드를 인식하지 못한다거나, 썬더볼트용 외장 디스플레이가 제대로 연결되지 않는 등의 문제가 있었다. 아크 컨트롤의 세부 메뉴도 로딩을 무한 반복한다거나, 공란으로 남아있는 등 미완성된 모습을 보여주었다. 드라이버가 업데이트될 수록 문제가 개선되겠지만, 아직은 갈 길이 멀어 보인다. 성능은 기대했던 수준이다. 가장 낮은 라인업의 제품임에도 엔비디아의 MX450M보다는 낫고, 생산성은 GTX 1650 Ti에 근접한 수준이다. 특히 GTX 1650 Ti는 별도로 방열 설계가 필요한 게이밍 노트북인데, 훨씬 더 가볍고 방열 성능이 적은 경량 노트북으로 따라잡았다는 점이 인상적이다. 추후 드라이버가 안정화된다면 훨씬 나은 수준을 보여줄 것으로 보인다. 인텔 아크 시리즈는 확실히 잘 만든 그래픽 카드다. 다듬어야 할 부분이 많지만, 엔비디아와 AMD 모두 간만에 등장한 경쟁에 긴장의 끈을 붙들어 매야 할 것으로 보인다.
"""
doc=doc_with_newlines.replace(".", "\n")
#한글, 영어, 숫자, +, -, 개행, 띄어쓰기만 놔두고 다 제거
#doc = re.sub(r"[^가-힣a-zA-Z0-9\+\-\n\s]","",doc)

#문장을 분리하였지만, 아직 앞뒤 공백이 있음
temp_texts =  doc.split('\n')
#앞뒤 공백이 제거된 문자열들이 저장되는 list
texts=[]
for text in temp_texts:
    text = text.strip()
    if text is not '':
        texts.append(text)

print(texts)

['[IT동아 남시현 기자] 지난 3월 31일, 인텔이 아크(Arc) 그래픽 카드 라인업을 공개하며 그래픽 카드 사업 진출을 선언했다', '2018년 6월에 외장 그래픽 카드 개발 여부를 공개한 지 4년 만의 결과다', '인텔의 외장 그래픽 카드는 인텔 Xe 마이크로 아키텍처를 기반으로 하며, 노트북용 그래픽 카드를 시작으로 올해 여름까지 데스크톱 제품까지 차례로 공개한다', '이로써 지난 20년 간 엔비디아와 AMD가 나눠갖던 그래픽 카드 시장은 이제 세 개 기업의 경쟁 구도로 경쟁하게 된다', '가장 처음 공개된 제품은 삼성전자 갤럭시 북2 프로에 탑재된 인텔 아크 A350M이다', 'A350M은 전체 노트북 그래픽 카드 라인업 중 가장 보급형인 제품이며, A350M을 시작으로 A370M, A550M, A730M, A770M의 순서로 성능이 향상된다', 'A 시리즈는 기존 GPU 벡터 장치 대비 16배가 향상된 인공지능 가속 기능 MXM AI 엔진과 최대 8K60p 12bit HDR 디코딩과 8K 10bit HDR 인코딩을 지원하는 Xe 미디어 엔진, 최대 두 대의 8K60p HDR 및 네 대의 4K 120p HDR 영상을 출력하는 Xe HPG 디스플레이 엔진을 탑재한다', '게이밍 역시 인공지능 기반의 해상도 향상 기능인 XeSS, 실시간 광선 추적 등이 적용된다', '갤럭시 북2 프로를 토대로 자세한 성능을 알아보자', '인텔 아크 A350M은 인텔 아크 A 시리즈 그래픽 카드 중 가장 보급형 그래픽카드로, 6개의 Xe 코어와 6개의 레이 트레이싱(실시간 광선 추적) 유닛, 4GB의 GDDR6 메모리가 적용된다', '노트북용 그래픽 카드 중 가장 최상급인 A770M은 32개의 Xe 코어 및 레이 트레이싱 유닛, 16GB의 GDDR6 메모리를 갖추고 있다', '그래픽 카드의 전력은 25~35W 수준으로 게이밍 노트북보다는 고성능 경량 노트북에 적합하지만, H265와 AV1, VP9에 대한 하드웨어 인코딩 및 디코딩을 지원해 가벼운 게임 혹은 영상 편집 용

In [5]:
# 품사 태깅
tokenized_doc = hannanum.pos(doc)
#print(tokenized_doc)
# 형태소 분석기를 통해 명사 추출한 문서를 만듦
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc
                   if (word[1] == 'N' or word[1] == 'F')
                   and not word[0].isdigit() 
                   and len(word[0])>1
                   and word[0] not in stop_words])

In [6]:
n_gram_range = (1, 2)
# CountVectorizer를 사용하여 n-gram을 추출(n_gram_range에 해당하는)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names()

In [7]:
#doc를 임베딩
doc_embedding = model.encode([doc])
#text들을 임베딩
texts_embedding = model.encode(texts)
#n_gram들을 임베딩
candidate_embeddings = model.encode(candidates)

In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


In [9]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [10]:
# nr_candidate가 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
mss_keywords=max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=15, nr_candidates=20)

In [11]:
# diversity값이 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
mmr_keywords=mmr(doc_embedding, candidate_embeddings, candidates, top_n=15, diversity=0.5)

In [12]:
#mmr을 이용해 문서와 관련높은 text를 10개 뽑아내고 해당에서 krwordrank를 이용하여 키워드 추출
new_texts = mmr(doc_embedding, texts_embedding, texts, top_n=15, diversity=0.1)
keywords = summarize_with_keywords(new_texts, min_count=min_count, max_length=max_length,
    beta=beta, max_iter=max_iter, stopwords=stop_words, verbose=True)

ten_wordrank_keywords=sorted(keywords, key=lambda x:x[1], reverse=True)[:30]

scan vocabs ... 
num vocabs = 175
done = 20


In [13]:
#전체 텍스트들을 문장 단위로 명사 형태소만 뽑아낸 뒤 krwordrank를 이용하여 키워드 추출
new_texts=[]
for text in texts:
    tokenized_text=hannanum.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_text
                       if (word[1] == 'N' or word[1] == 'F')
                       and not word[0].isdigit() 
                       and len(word[0])>1
                       and word[0] not in stop_words])
    new_texts.append(tokenized_nouns)

keywords = summarize_with_keywords(new_texts, min_count=min_count, max_length=max_length,
    beta=beta, max_iter=max_iter, stopwords=stop_words, verbose=True)

wordrank_keywords=sorted(keywords, key=lambda x:x[1], reverse=True)[:30]

scan vocabs ... 
num vocabs = 618
done = 20


In [14]:
print("mss_keywords: ",mss_keywords)
print()
print("mmr_keywords: ",mmr_keywords)
print()
print("ten_wordrank_keywords: ",ten_wordrank_keywords)
print()
print("wordrank_keywords: ",wordrank_keywords)

mss_keywords:  ['변환 인텔', '그래픽 칩셋', '여름 데스크톱', '1240p 인텔', '보급형 그래픽카드', '카드 인텔', '제작 인텔', '그래픽 인텔', '칩셋 스펙', '1천352점 인텔', '12세대 인텔', '인텔 공개', '인텔 cpu', '인텔 아이리스', 'a350m 인텔']

mmr_keywords:  ['a350m 인텔', '선언 2018년', '사업 진출', '여름 데스크톱', '프로세서 12세대', '보급형 그래픽카드', '마지막 오픈월드', '모바일 타임스파', 'mx450보다 1천800점', '인공지능 가속', '생태계 시작', 'gtx 2gb', '기반 테스트', 'arc 그래픽', '제품 삼성전자']

ten_wordrank_keywords:  ['제품', '스펙을', '노트북', '인텔', '아크', '경쟁', '탑재', '가장', '외장', '시작으로', '게임', '확인', '라인업', '코어', '최신', '점수는', '삼성전자', '기반', '프로', '갤럭시', '그래픽', '카드', '성능', '지난', '보급형', '6개의', '공개', 'Xe', 'AMD', 'A350M']

wordrank_keywords:  ['호환성', '실행', '수행', '적합', '소프', '제품', '지포스', '노트북', '인텔', '토탈워:워해머', '아크', '인코딩', '디코딩', '벤치마크', '편집', '엔진', '수준', '설정', '경쟁', '탑재', '외장', '내장', '시작', '게임', '타임스파', '확인', '라인업', '게이밍', '파이', '하이퍼']
